In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import vincent
%matplotlib inline

In [2]:
df = pd.read_csv('./data/RustSurvey+EarthEngine.csv')

/Users/arokem/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (11,20,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(8304, 886)

In [4]:
list(df.columns)

['Location.ID',
 'HostGenusID',
 'HostGenusNameScientific',
 'HostSpeciesID',
 'HostSpeciesNameScientific',
 'HostCultivarName',
 'UserInit',
 'ObsYear',
 'ObsDate',
 'SurveySiteID',
 'SurveySiteName',
 'SurveyorName',
 'InstitutionName',
 'LocationName',
 'Latitude',
 'Longitude',
 'Altitude',
 'FieldArea',
 'GrowthStageID',
 'GrowthStageName',
 'Comment',
 'StemRust.Severity',
 'StemRust.SeverityName',
 'StemRust.Incidence',
 'StemRust.IncidenceName',
 'StemRust.InfectionType',
 'StemRust.InfectionTypeName',
 'LeafRust.Severity',
 'LeafRust.SeverityName',
 'LeafRust.Incidence',
 'LeafRust.IncidenceName',
 'LeafRust.InfectionType',
 'LeafRust.InfectionTypeName',
 'YellowRust.Severity',
 'YellowRust.SeverityName',
 'YellowRust.Incidence',
 'YellowRust.IncidenceName',
 'YellowRust.InfectionType',
 'YellowRust.InfectionTypeName',
 'StemRust.Binary',
 'YellowRust.Binary',
 'NoRust.Binary',
 'GLCF_GLS_TCC_source_index',
 'GLCF_GLS_TCC_tree_canopy_cover',
 'GLCF_GLS_TCC_uncertainty',
 'LAND

In [5]:
df['ObsYear'].unique()

array([2012, 2013, 2014, 2007, 2008, 2009, 2010, 2011, 2015, 2016])

In [6]:
# Using Tableau ten hex from: https://community.tableau.com/message/200964
year_colors = {2007:'#1F77B4', 
               2008:'#FF7F0E', 
               2009:'#2CA02C', 
               2010:'#D62728', 
               2011:'#9467BD',
               2012:'#8C564B',
               2013:'#CFECF9', 
               2014:'#7F7F7F',
               2015:'#BCBD22', 
               2016:'#17BECF'}

In [7]:
severity_colors = {0:'#4EE500' , 1:'#879F05', 2:'#C05A0A', 3:'#F91510', np.nan:'#FFFFFF'}

In [8]:
folium.Popup()

In [9]:
list(df.columns)

['Location.ID',
 'HostGenusID',
 'HostGenusNameScientific',
 'HostSpeciesID',
 'HostSpeciesNameScientific',
 'HostCultivarName',
 'UserInit',
 'ObsYear',
 'ObsDate',
 'SurveySiteID',
 'SurveySiteName',
 'SurveyorName',
 'InstitutionName',
 'LocationName',
 'Latitude',
 'Longitude',
 'Altitude',
 'FieldArea',
 'GrowthStageID',
 'GrowthStageName',
 'Comment',
 'StemRust.Severity',
 'StemRust.SeverityName',
 'StemRust.Incidence',
 'StemRust.IncidenceName',
 'StemRust.InfectionType',
 'StemRust.InfectionTypeName',
 'LeafRust.Severity',
 'LeafRust.SeverityName',
 'LeafRust.Incidence',
 'LeafRust.IncidenceName',
 'LeafRust.InfectionType',
 'LeafRust.InfectionTypeName',
 'YellowRust.Severity',
 'YellowRust.SeverityName',
 'YellowRust.Incidence',
 'YellowRust.IncidenceName',
 'YellowRust.InfectionType',
 'YellowRust.InfectionTypeName',
 'StemRust.Binary',
 'YellowRust.Binary',
 'NoRust.Binary',
 'GLCF_GLS_TCC_source_index',
 'GLCF_GLS_TCC_tree_canopy_cover',
 'GLCF_GLS_TCC_uncertainty',
 'LAND

In [26]:
def get_line_plot(row): 
    data_dict = {'Band': range(7)}
    for TOA in range(1, 13):
        data_dict['LANDSAT_LE7_L1T_32DAY_TOA_%02d'%TOA] = list(row[['LANDSAT_LE7_L1T_32DAY_TOA_%02d_B1'%TOA,
                                                                     'LANDSAT_LE7_L1T_32DAY_TOA_%02d_B2'%TOA,
                                                                     'LANDSAT_LE7_L1T_32DAY_TOA_%02d_B3'%TOA,
                                                                     'LANDSAT_LE7_L1T_32DAY_TOA_%02d_B4'%TOA,
                                                                     'LANDSAT_LE7_L1T_32DAY_TOA_%02d_B5'%TOA,
                                                                     'LANDSAT_LE7_L1T_32DAY_TOA_%02d_B7'%TOA,
                                                                     'LANDSAT_LE7_L1T_32DAY_TOA_%02d_B8'%TOA]])

    lines = vincent.Line(data_dict, iter_idx='Band', width=420, height=500)
    lines.legend(title='Categories')
    lines.axis_titles(x='Band', y='Reflectance')
    vega= folium.Vega(json.loads(lines.to_json()), width=700, height=550)      
    return vega

In [ ]:
for year, year_df in df.groupby('ObsYear'):
    year_map = folium.Map(location=[df['Latitude'].iloc[0], df['Longitude'].iloc[0]], zoom_start=6)
    for idx in range(year_df.shape[0]):
        row = year_df.iloc[idx]
        lon = row['Latitude']
        lat = row['Longitude']
        if np.isnan(row['StemRust.Severity']):
            c = 'FFFFFF'
        else:
            c = severity_colors[row['StemRust.Severity']]

        vega = get_line_plot(row)
        marker = folium.CircleMarker([lon, lat], 
                                     color=c,
                                     fill_color=c,
                                     radius=1000,
                                     popup=folium.Popup(max_width=700).add_child(vega))
        marker.add_to(year_map)

    year_map.save('ethiopia%s.html' % year)

In [30]:
year_map